In [1]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from qiskit.primitives import Sampler,Estimator
from qiskit import QuantumCircuit
from qiskit.circuit import QuantumRegister,ClassicalRegister
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2,RealAmplitudes
from qiskit.algorithms.gradients import ParamShiftEstimatorGradient,ParamShiftSamplerGradient
%matplotlib inline


/var/folders/9v/dz00k5396wl369lhxwggy12m0000gn/T/ipykernel_2428/2958622593.py:9: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.gradients import ParamShiftEstimatorGradient,ParamShiftSamplerGradient


In [2]:
from VQS_header import compute_params_list,compute_magnetization

In [246]:
num_qubits = 2
j_coupling = 1/4
h_coupling = 1

num_layers = 2
time_max = 5e0
epsilon_det = 1e-7
num_steps = 100
sampler = Sampler()
estimator = Estimator()
nShots=None

dt=time_max/num_steps
sampler=Sampler()

print("dt = ",dt)

dt =  0.05


In [253]:
def addSuzukiTrotter(circuit,j,h,dt):
    num_qubits=circuit.num_qubits
    #Implementing Suzuki-Trotter evolution step for the transverse Ising
    for i in range(num_qubits):
        circuit.rx(-2*h*dt,i)
    
    for i in range(num_qubits-1):
        circuit.rzz(-2*j*dt,i,i+1)
    return circuit
    

def getCircuit(num_qubits,n_reps=3,j=j_coupling,h=h_coupling,dt=dt):
    circuit = QuantumCircuit(num_qubits,num_qubits) 
    
    ansatz =RealAmplitudes(num_qubits=num_qubits,reps=n_reps,parameter_prefix='ansatz_last')
    ansatz_inv =RealAmplitudes(num_qubits=num_qubits,reps=n_reps,parameter_prefix='ansatz_current').inverse()
    #circuit.h(circuit.qubits)
    circuit.compose(ansatz_inv, inplace=True)
    addSuzukiTrotter(circuit,j,h,dt)
    circuit.compose(ansatz, inplace=True)  
    
    circuit.measure(circuit.qubits,circuit.clbits)
    return circuit

In [254]:
qc = getCircuit(num_qubits,n_reps=num_layers)

In [255]:
# qc.draw('mpl')

In [263]:
print("dt = ",dt)
n_params_ansatz=int(qc.num_parameters/2)
weights_init=np.random.rand(n_params_ansatz)
weights_init=np.concatenate([weights_init,weights_init])
weights=weights_init
#learning_rate=7e0
learning_rate=1e0

for step in range(50):
    weights_list=[weights]
    for i in range(n_params_ansatz):
        weightsPlus=weights.copy()
        weightsPlus[i]+=np.pi/2
        weights_list.append(weightsPlus)

        weightsMinus=weights.copy()
        weightsMinus[i]-=np.pi/2
        weights_list.append(weightsMinus)

    results=sampler.run(circuits=[qc]*len(weights_list),parameter_values=weights_list).result().quasi_dists
    output=results[0][0]
    #loss=(1-(output)**2)/(dt**2)
    loss=(1-output)**2

    grad_list=[]
    for i in range(n_params_ansatz):

        tmpGrad_output=0.5*(results[2*i+1][0] - results[2*i+2][0])
        #tmpGrad = ((-2*output)/(dt**2))*tmpGrad_output
        tmpGrad = ((-2*output))*tmpGrad_output
        #tmpGrad = tmpGrad_output*(-1)
        grad_list.append(tmpGrad)
        
    grad_list=np.array(grad_list)
    weights[0:n_params_ansatz]-=learning_rate*grad_list
   
    print(output,loss)
    print('----')

dt =  0.05
0.9986300224133644 1.876838587883891e-06
----
0.9986302872176702 1.8761131060775857e-06
----
0.9986264789403572 1.886560101282376e-06
----
0.9986180766017124 1.909712278734684e-06
----
0.99859991336359 1.9602425894538755e-06
----
0.9985600949065168 2.073326678238979e-06
----
0.9984710374703932 2.337726416941533e-06
----
0.998266420979027 3.00529622195785e-06
----
0.9977813819193596 4.9222661877444426e-06
----
0.996587527384242 1.1644969353298624e-05
----
0.9935623759183974 4.1443003816030285e-05
----
0.9857051152997302 0.00020434372859400902
----
0.9661178085765024 0.0011480028956585328
----
0.9225298358647744 0.006001626331138789
----
0.8627811570730562 0.018829010854209263
----
0.8266717685786007 0.030042675807670154
----
0.8441099080554474 0.02430172076648107
----
0.8303092217948052 0.02879496020788463
----
0.8484125998731543 0.022978739877216416
----
0.8353082100961602 0.027123385661730514
----
0.8513346785384357 0.02210137780527025
----
0.839813070329259 0.0256598524373

In [251]:
len(weights_list)

13

In [222]:
n_params_ansatz

6

In [ ]:
gradSampler=ParamShiftSamplerGradient(sampler)

In [ ]:
# qc = getCircuit(num_qubits,n_reps=3,dt=0.05)
# learning_rate=1e1


# weights_init=np.random.rand(int(qc.num_parameters/2))
# weights_last=weights_init.copy() # important to keep it as a copy
# weights_tmp=weights_init.copy()
# weights_all=np.concatenate([weights_tmp,weights_last])

# print(weights_all)

In [ ]:

# for step in range(2):

#     output=sampler.run(circuits=[qc],parameter_values=[weights_all]).result().quasi_dists[0][0]
#     loss=((1-output)**2)/dt
    
#     print("initial loss =",loss)

#     for i in range(100):
        
#         if loss<1e-5:
#             print("loss = ",loss, "number of iterations = ",i)
#             break
            
#         result=gradSampler.run(circuits=[qc],parameter_values=[weights_all],parameters=[qc.parameters[0:int(qc.num_parameters/2)]])
#         grad=result.result().gradients[0]
#         for j in range(len(weights_tmp)):
#             #weights_tmp[j]+=learning_rate*grad[j][0]
#             weights_tmp[j]-=2*((1-output)/dt)*(-1)*learning_rate*grad[j][0]
#         weights_all=np.concatenate([weights_tmp,weights_last])
        

#         output=sampler.run(circuits=[qc],parameter_values=[weights_all]).result().quasi_dists[0][0]
#         loss=((1-output)**2)/dt
#         print(loss)

#     print("loss = ",loss, "number of iterations = ",100)
            

    
#     weights_last=weights_tmp.copy()
#     weights_all=np.concatenate([weights_tmp,weights_last])
    
